In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:
def assemble():
    df = pd.DataFrame(
        columns=["Title", "Size", "Price", "BOGO", "Date Range"]
    )
    driver = webdriver.Chrome()
    url = "https://www.publix.com/d/all-categories/c9074869-795e-4e35-84c1-550b104e8618"
    num_pages = get_num_pages(driver, f"{url}?page={1}")
    
    for i in range(1, 5+1): # int(num_pages)+1
        cards = scrape_product_cards(driver, f"{url}?page={i}")
        button = driver.find_element(By.XPATH, "//*[@id=\"main\"]/div[3]/div/div[2]/div[1]/div[2]/nav[1]/button[9]")
        button.click()
        cards_to_df(cards, df)
    
    driver.quit()
    return df


def get_num_pages(driver, url):
    driver.get(url)

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'button-section'))
        )
    except:
        driver.quit()
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    return (soup.find_all(class_="toggle-button"))[-1].get_text(strip=True)


def scrape_product_cards(driver, url):
    driver.get(url)

    try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'button-section'))
        )
    except:
        driver.quit()

    soup = BeautifulSoup(driver.page_source, 'lxml')
    cards = soup.find_all(class_="p-grid-item")
    return cards


def cards_to_df(cards, df):
    for card in cards:
        title = card.find(class_="title-wrapper")
        size = card.find(class_="size")
        price = card.find(class_="price")
        bogo = card.find(class_="p-savings-badge savings-badge bogo")
        date_range = card.find(class_="p-price-badge__date-range")

        df.loc[-1] = [
            title.get_text(strip=True),
            size.get_text(strip=True) if size else None,
            price.get_text(strip=True) if price else None,
            True if bogo else False,
            date_range.get_text(strip=True) if date_range else None
            ]
        df.index = df.index + 1
        df.sort_index(inplace=True)


res = assemble()
display(res)

,Title,Size,Price,BOGO,Date Range
0,"Publix Deli Lemonade, Old-Fashioned Half Gallon",None,None,False,None
1,Honeycrisp Apples Large,None,None,False,None
2,"Little Debbie Breakfast Pastries, Honey Buns",6 pastries [10.6 oz (301 g)],None,False,None
3,"Progresso Broth, Chicken, Classic",32 oz (2 lb) 907 g,None,False,None
4,"Lay's Potato Chips, Honey Barbecue Flavored",7.75 oz (219.7 g),None,False,None
...,...,...,...,...,...
235,Publix Deli 8-Piece Mixed Fried Chicken,None,None,False,None
236,"Eternal Spring Water, Naturally Alkaline",1.5 liter (50.7 fl oz),None,False,None
237,"Nature's Own Nature's Own Honey Wheat, Honey W...",20 oz (1 lb 4 oz) 567 g,None,False,None
238,Bananas,None,None,False,None
